In [1]:
import pyomo

# instance_io tests

In [2]:
from instance_io import *

In [3]:
read_max_sc_qbf_instance("data/sample_instances/1.txt")

(5,
 [{1, 2}, {2, 3, 4}, {1, 4}, {3, 5}, {4, 5}],
 [[3.0, 1.0, -2.0, 0.0, 3.0],
  [0.0, -1.0, 2.0, 1.0, -1.0],
  [0.0, 0.0, 2.0, -2.0, 4.0],
  [0.0, 0.0, 0.0, 0.0, 5.0],
  [0.0, 0.0, 0.0, 0.0, 3.0]])

# gurobi + pyomo test

In [4]:
from pyomo.opt import SolverFactory

import pyomo.environ as pyo

# Create a simple test model
test_model = pyo.ConcreteModel()

# Add variables
test_model.x = pyo.Var(bounds=(0, None))
test_model.y = pyo.Var(bounds=(0, None))

# Add objective
test_model.obj = pyo.Objective(expr=2*test_model.x + test_model.y, sense=pyo.maximize)

# Add constraints
test_model.constraint1 = pyo.Constraint(expr=test_model.x + test_model.y <= 10)
test_model.constraint2 = pyo.Constraint(expr=2*test_model.x + test_model.y <= 15)

# Test Gurobi solver
try:
    gurobi_solver = SolverFactory('gurobi')
    if gurobi_solver.available():
        print("✓ Gurobi solver is available")
        
        # Solve the test problem
        result = gurobi_solver.solve(test_model, tee=False)
        
        if result.solver.termination_condition == pyo.TerminationCondition.optimal:
            print("✓ Gurobi solved the test problem successfully")
            print(f"  Optimal value: {pyo.value(test_model.obj)}")
            print(f"  x = {pyo.value(test_model.x)}")
            print(f"  y = {pyo.value(test_model.y)}")
        else:
            print("✗ Gurobi failed to solve the test problem")
    else:
        print("✗ Gurobi solver is not available")
        
except Exception as e:
    print(f"✗ Error testing Gurobi: {e}")

print("\n✓ Pyomo is properly installed and working")

✓ Gurobi solver is available
✓ Gurobi solved the test problem successfully
  Optimal value: 15.0
  x = 7.5
  y = 0.0

✓ Pyomo is properly installed and working


# Solving instances

In [29]:
n, sets, A = read_max_sc_qbf_instance("data/sample_instances/3.txt")
display(n, sets, A)

universe = set.union(*sets)
print(f"Universe: {universe}")

4

[{1}, {2, 3}, {1, 4}, {3, 4}]

[[0.0, -2.0, 1.0, 0.0],
 [0.0, 4.0, 2.0, 3.0],
 [0.0, 0.0, 5.0, -1.0],
 [0.0, 0.0, 0.0, 2.0]]

Universe: {1, 2, 3, 4}


In [30]:
from pyomo.environ import ConcreteModel, Var, RangeSet

model = pyo.ConcreteModel()

# Vars:
model.I = RangeSet(n)
model.x = Var (model.I, domain=pyo.Binary)

# Objective:
def objective_rule(model):
    return sum(model.x[i] * A[i-1][j-1] * model.x[j] for i in model.I for j in model.I)

model.obj = pyo.Objective(rule=objective_rule, sense=pyo.maximize)

# Constraints:
model.set_cover_contraints = pyo.ConstraintList()
for element in universe:
    # Get all indexes from the sets that contain element:
    containing_sets_idx: List[int] = []
    for i, set in enumerate(sets):
        if element in set:
            containing_sets_idx.append(i)
    
    # print(f"Element: {element}, csi: {containing_sets_idx}")
    
    model.set_cover_contraints.add(expr=sum(model.x[i+1] for i in containing_sets_idx) >= 1)

In [31]:
opt = SolverFactory('gurobi')
opt.solve(model)

{'Problem': [{'Name': 'x1', 'Lower bound': 15.0, 'Upper bound': 15.0, 'Number of objectives': 1, 'Number of constraints': 4, 'Number of variables': 4, 'Number of binary variables': 4, 'Number of integer variables': 4, 'Number of continuous variables': 0, 'Number of nonzeros': 7, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '0.0008060932159423828', 'Error rc': 0, 'Time': 0.08958649635314941}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [32]:
print("Solution values:")
for i in model.I:
    print(f"x[{i}] = {pyo.value(model.x[i])}")
print(f"Objective function value: {pyo.value(model.obj)}")

Solution values:
x[1] = 0.0
x[2] = 1.0
x[3] = 1.0
x[4] = 1.0
Objective function value: 15.0
